# BASIC CRUD OPERATIONS

In [13]:
import pandas as pd
from datetime import datetime
from pymongo import MongoClient
# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['eduhub_db']

In [2]:
print("Collections in eduhub:")
print(db.list_collection_names())

Collections in eduhub:
['lessons', 'submissions', 'assignments', 'courses', 'users', 'enrollments']


## CREATE OPERATIONS

In [ ]:
#Adding a new student user
new_student = {
    "userId": "STU016",   
    "email": "student16@example.com",
    "firstName": "Ada",
    "lastName": "Eze",
    "role": "student",
    "dateJoined": datetime.now(),
    "profile": {
        "bio": "Passionate learner exploring data science and analytics.",
        "avatar": "https://example.com/avatar16.png",
        "skills": ["Python", "SQL"]
    },
    "isActive": True
}
db.users.insert_one(new_student)
print("Added new student")



# Creating a new course
new_course = {
    "courseId": "CRS009",   
    "title": "Data Visualization with Power BI",
    "description": "Learn to design and develop interactive dashboards with Power BI.",
    "instructorId": "INST001",   
    "category": "Data Science",
    "level": "beginner",   
    "duration": 40,        
    "price": 99.99,
    "tags": ["data", "visualization", "business-intelligence"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now(),
    "isPublished": True
}
db.courses.insert_one(new_course)
print("Added new course")



#Enrolling a student in a course
new_enrollment = {
    "enrollmentId": "ENR016",   
    "studentId": "STU016",      
    "courseId": "CRS009",       
    "enrolledAt": datetime.now(),
    "status": "active"          
}
db.enrollments.insert_one(new_enrollment)
print("Enrolled student in course")



#Adding a new lesson to an existing course
new_lesson = {
    "lessonId": "LES026",    
    "courseId": "CRS009",    
    "title": "Getting Started with Power BI",
    "content": "This lesson introduces the Power BI interface and its core features.",
    "order": 1,              
    "resources": ["https://docs.microsoft.com/en-us/power-bi/"],
    "createdAt": datetime.now(),
    "updatedAt": datetime.now()
}
db.lessons.insert_one(new_lesson)
print("Added new lesson to course")

Added new student
Added new course
Enrolled student in course
Added new lesson to course


## READ OPERATIONS


### Active Students 

In [ ]:
#Finding all active student
active_students = db.users.find(
    {"role": "student", "isActive": True},
    {"_id": 0, "userId": 1, "firstName": 1, "lastName": 1, "email": 1}
)

students_df = pd.DataFrame(list(active_students))
print("Active Students:")
students_df[['userId', 'firstName', 'lastName', 'email']].head()

Active Students:


,userId,firstName,lastName,email
0,STU001,Melissa,Powell,mosstara@example.net
1,STU003,Alexandra,Carr,mercersusan@example.org
2,STU004,Amy,Walton,rwhite@example.org
3,STU008,Katie,Snow,ulane@example.org
4,STU009,Elizabeth,Kim,wgeorge@example.org


### Retrieving Course Details with Instructor Information

In [ ]:
# Retreving course details with instructors information
course_with_instructor = db.courses.aggregate([
    {
        "$lookup": {
            "from": "users",
            "localField": "instructorId",
            "foreignField": "userId",
            "as": "instructor"
        }
    },
    { "$unwind": "$instructor" },  
    {
        "$project": {
            "_id": 0,
            "courseId": 1,
            "title": 1,
            "category": 1,
            "level": 1,
            "instructor.firstName": 1,
            "instructor.lastName": 1,
            "instructor.email": 1
        }
    }
])

# Convert cursor to list and then to DataFrame
courses_df = pd.DataFrame(list(course_with_instructor))

# Display the DataFrame
print("Courses with Instructor Info:")


  courseId                             title         category         level  \
0   CRS001    Machine Learning for Beginners     Data Science  intermediate   
1   CRS002      Introduction to Data Science     Data Science      beginner   
2   CRS003    Machine Learning for Beginners     Data Science  intermediate   
3   CRS004  Backend Development with Node.js  Web Development      advanced   
4   CRS005          Python for Data Analysis     Data Science      beginner   

                                          instructor  
0  {'email': 'ucastaneda@example.org', 'firstName...  
1  {'email': 'dcarlson@example.org', 'firstName':...  
2  {'email': 'john04@example.org', 'firstName': '...  
3  {'email': 'ucastaneda@example.org', 'firstName...  
4  {'email': 'john04@example.org', 'firstName': '...  
Courses with Instructor Info:


###  Getting all Courses in a Specific Category

In [ ]:
#Getting all courses in a specific category
category = "Data Science"

courses_in_category = db.courses.find(
    {"category": category},
    {"_id": 0, "courseId": 1, "title": 1, "level": 1}
)

print(f"Courses in {category}:")
for course in courses_in_category:
    print(course)

Courses in Data Science:
{'courseId': 'CRS009', 'title': 'Data Visualization with Power BI', 'level': 'beginner'}


### Finding Students Enrolled in a Particular Course


In [ ]:
# Finding student enrolled in a particular course
course_id = "CRS001"

students_in_course = db.enrollments.aggregate([
    { "$match": {"courseId": course_id} },
    {
        "$lookup": {
            "from": "users",
            "localField": "studentId",
            "foreignField": "userId",
            "as": "student"
        }
    },
    { "$unwind": "$student" },
    {
        "$project": {
            "_id": 0,
            "student.userId": 1,
            "student.firstName": 1,
            "student.lastName": 1,
            "student.email": 1
        }
    }
])

print(f"Students enrolled in {course_id}:")
for student in students_in_course:
    print(student)

Students enrolled in CRS001:
{'student': {'userId': 'STU011', 'email': 'john87@example.net', 'firstName': 'Autumn', 'lastName': 'Browning'}}
{'student': {'userId': 'STU006', 'email': 'antonio63@example.com', 'firstName': 'Nicole', 'lastName': 'Jones'}}


### Searching Courses by Title (case-insensitive, partial match)

In [ ]:
# Searching courses by Title
search_term = ".*python.*"
matched_courses = db.courses.find(
    {"title": {"$regex": search_term, "$options": "i"}},
    {"_id": 0, "courseId": 1, "title": 1, "category": 1, "level": 1}
)

print(f"Courses matching '{search_term}':")
for course in matched_courses:
    print(course)

Courses matching '.*python.*':
{'courseId': 'CRS005', 'title': 'Python for Data Analysis', 'category': 'Data Science', 'level': 'beginner'}
{'courseId': 'CRS008', 'title': 'Python for Data Analysis', 'category': 'Data Science', 'level': 'advanced'}


## UPDATE OPERATIONS

### Update a User’s Profile Information

In [4]:
#Updating a Useers Profile information
user_id = "STU005"

update_result = db.users.update_one(
    {"userId": user_id},
    {
        "$set": {
            "profile.bio": "Enthusiastic learner with interest in AI and Machine Learning.",
            "profile.avatar": "https://example.com/new_avatar.png",
            "profile.skills": ["Python", "SQL", "Machine Learning"]
        }
    }
)

print(f"Matched: {update_result.matched_count}, Modified: {update_result.modified_count}")

Matched: 1, Modified: 1


### Mark a Course as Published

In [5]:
# Marking a course as Published
course_id = "CRS003"

update_result = db.courses.update_one(
    {"courseId": course_id},
    {"$set": {"isPublished": True, "updatedAt": datetime.now()}}
)

print(f"Course {course_id} marked as published.")

Course CRS003 marked as published.


### Update Assignment Grades

In [7]:
#Updating Assignment Grades
submission_id = "SUB010"

update_result = db.submissions.update_one(
    {"submissionId": submission_id},
    {
        "$set": {
            "grade":80,
            "feedback": "Excellent work, keep it up!",
            "gradedAt": datetime.now()
        }
    }
)

print(f"Updated grade for submission {submission_id}")

Updated grade for submission SUB010


### Add Tags to an Existing Course

In [9]:
#Adding Tags to an Existing Course
course_id = "CRS005"

update_result = db.courses.update_one(
    {"courseId": course_id},
    {"$addToSet": {"tags": {"$each": ["AI", "Data Science", "Python"]}}}
)

print(f"Tags updated for course {course_id}")

Tags updated for course CRS005


## DELETE OPERATIONS

### Remove a User (soft delete by setting isActive to false)


In [ ]:
# Removing user



### Deleting  an Enrollment

In [11]:
#Deleting an Enrollment
enrollment_id = "ENR005"

delete_result = db.enrollments.delete_one({"enrollmentId": enrollment_id})

print(f"Deleted {delete_result.deleted_count} enrollment(s) with ID {enrollment_id}")

Deleted 1 enrollment(s) with ID ENR005


###  Remove a Lesson from a Course

In [10]:
# Removing a lesson from a Course
lesson_id = "LES008"

delete_result = db.lessons.delete_one({"lessonId": lesson_id})

print(f"Deleted {delete_result.deleted_count} lesson(s) with ID {lesson_id}")

Deleted 1 lesson(s) with ID LES008
